In [169]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_predict, cross_validate, GridSearchCV

from sklearn.preprocessing import RobustScaler, PowerTransformer

import itertools

from sklearn.metrics.pairwise import paired_distances

import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from sklearn.compose import ColumnTransformer
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [170]:
df = pd.read_excel('/home/guest/Загрузки/Basheer_demo.xlsx')

In [137]:
[x.replace('l0_', '') for x in df.columns if 'l0_' in x][:10]
df[[x for x in df.columns if 'l0_' in x][:10]]

,l0_42_03257085331836,l0_43_04741898699762,l0_44_01450529149254,l0_44_0489926172728,l0_44_99144370755021,l0_45_99197266970869,l0_47_04077398412186,l0_48_045099177099885,l0_49_00477743760921,l0_49_99501251131184
0,0.067512,0.628985,0.079979,0.065705,3.712502,0.065790,0.103783,0.005272,0.046767,0.007219
1,0.052837,0.293217,0.041497,0.027165,3.342605,0.056691,0.089783,0.002514,0.008929,0.004855
2,0.034711,0.289125,0.039634,0.023119,3.528768,0.058900,0.210027,0.006351,0.027325,0.005452
3,0.065292,0.965013,0.055779,0.059098,2.963412,0.052547,0.143320,0.004360,0.017409,0.006269
4,0.080594,3.142379,0.052536,0.122263,2.635041,0.043910,0.100054,0.002831,0.004470,0.004573
...,...,...,...,...,...,...,...,...,...,...
75,0.070222,1.218192,0.060510,0.058470,3.800003,0.064128,0.162522,0.004768,0.066721,0.009513
76,0.083748,2.472044,0.110516,0.126304,2.979950,0.058456,8.012591,0.178858,0.030891,0.008964
77,0.059168,2.266683,0.108345,0.127781,1.556364,0.071685,0.186592,0.005847,0.043078,0.005111
78,0.288774,1.230780,0.062873,0.050527,1.983028,0.051258,0.217254,0.006029,0.002695,0.002760


In [171]:
vocs_names = [x.replace('l0_', '') for x in df.columns if 'l0_' in x]#[:5]

df_vocs = df[[x for x in df.columns if 'l0_' in x]]#[:5]]
df_vocs.columns = vocs_names

unique_probe_id = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['PatientID'] = df['Cardio_Qvark_ID']
df_vocs['ProbeId'] = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['DateTime'] = [datetime.now() for x in range(df.shape[0])]

y = (df['Myocardial_perfusion_defect_after_stress_ATP'] == 'Yes').astype(float)
df_vocs['target'] =y.replace([0, 1], ['Healthy', 'ИБС'])

X_train, _, y_train, _ = train_test_split(df_vocs.drop('target', axis = 'columns'), y, test_size=0.01, random_state=42)
X_train['target_short_name'] =y_train.replace([0, 1], ['Healthy', 'ИБС'])
X_train['target_icd_10'] =y_train.replace([0, 1], ['Healthy', 'I20.8'])
X_train['target_icd_11'] =y_train.replace([0, 1], ['Healthy', 'BA40.1'])

df_vocs = X_train

df1 = df_vocs

print(df_vocs.to_markdown())

|    |   42_03257085331836 |   43_04741898699762 |   44_01450529149254 |   44_0489926172728 |   44_99144370755021 |   45_99197266970869 |   47_04077398412186 |   48_045099177099885 |   49_00477743760921 |   49_99501251131184 |   51_03878893892946 |   51_93582662455944 |   53_03690075569887 |   55_03975620678352 |   55_93512749697764 |   56_05352984794852 |   57_06920514918207 |   58_06961588414241 |   58_95512627855279 |   59_05781995045748 |   60_05530483707332 |   61_03262621334451 |   62_028803351046605 |   63_02131475672599 |   64_02530160379577 |   65_02586759164836 |   67_05677308444703 |   68_06320828624578 |   69_07278324575172 |   70_07566918909677 |   71_05402913977443 |   72_05367879285716 |   73_06505607160392 |   74_05798179492636 |   75_04653703736516 |   75_94813807735272 |   76_04942842237271 |   77_0538237072059 |   79_05616364051586 |   81_07303293978276 |   82_07592688169181 |   83_08439935902163 |   85_06482451619402 |   87_07684847044642 |   87_93367781464133 |   8

In [174]:
vocs_names = [x.replace('l0_', '') for x in df.columns if 'l0_' in x][4:10]

df_vocs = df[[x for x in df.columns if 'l0_' in x][4:10]]
df_vocs.columns = vocs_names

unique_probe_id = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['PatientID'] = df['Cardio_Qvark_ID']
df_vocs['ProbeId'] = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['DateTime'] = [datetime.now() for x in range(df.shape[0])]

y = (df['Myocardial_perfusion_defect_after_stress_ATP'] == 'Yes').astype(float)
df_vocs['target'] =y.replace([0, 1], ['Healthy', 'ИБС'])

X_train, _, y_train, _ = train_test_split(df_vocs.drop('target', axis = 'columns'), y, test_size=.98, random_state=58)
X_train['target_short_name'] =y_train.replace([0, 1], ['Healthy', 'ИБС'])
X_train['target_icd_10'] =y_train.replace([0, 1], ['Healthy', 'I20.8'])
X_train['target_icd_11'] =y_train.replace([0, 1], ['Healthy', 'BA40.1'])

df_vocs = X_train

df2 = df_vocs

print(df_vocs.to_markdown())

|    |   44_99144370755021 |   45_99197266970869 |   47_04077398412186 |   48_045099177099885 |   49_00477743760921 |   49_99501251131184 |   PatientID | ProbeId    | DateTime                   | target_short_name   | target_icd_10   | target_icd_11   |
|---:|--------------------:|--------------------:|--------------------:|---------------------:|--------------------:|--------------------:|------------:|:-----------|:---------------------------|:--------------------|:----------------|:----------------|
| 35 |             3.97766 |           0.0668665 |            0.130496 |           0.00457821 |           0.0493458 |          0.00639664 |       17501 | ProbeId_35 | 2024-08-31 10:10:39.214826 | Healthy             | Healthy         | Healthy         |


In [175]:
import json
from datetime import datetime
from typing import List, Optional, Union
from dataclasses import dataclass, field
import pandas as pd

@dataclass
class VOC:
    identifier: str = 'unknown'
    value: Optional[float] = None
    method: str = 'Silantiev_maximization'
    chemical_formula: Optional['Substance'] = None

@dataclass
class Substance:
    voc_identifiers: Union[str, List[str]] = 'unknown'
    breath_type: str = 'any'
    chemical_formula: Optional[str] = 'unknown'

@dataclass
class Target:
    short_name: str
    icd_10: str
    icd_11: str

@dataclass
class Person:
    person_id: str

@dataclass
class Volatome:
    unique_id: str
    person: Person  # Use the Person class instead of a string for person_id
    collection_time: datetime = field(default_factory=datetime.now)
    breath_type: str = 'normal'
    normalization: Optional[str] = 'raw'
    targets: List[Target] = field(default_factory=list)
    voc_objects: List[VOC] = field(default_factory=list)

    @property
    def shape(self) -> int:
        return len(self.voc_objects)
    

@dataclass
class VolatomeVoid:
    volatome_storage: dict = field(default_factory=dict)

    def add_volatome(self, volatome: Volatome):
        self.volatome_storage[volatome.unique_id] = volatome

    def get_volatome(self, unique_id: str) -> Optional[Volatome]:
        return self.volatome_storage.get(unique_id, None)

    def remove_volatome(self, unique_id: str):
        if unique_id in self.volatome_storage:
            del self.volatome_storage[unique_id]

    def to_json(self, filename: str):
        with open(filename, 'w') as f:
            json.dump({uid: volatome.__dict__ for uid, volatome in self.volatome_storage.items()}, f, default=str)

    @property
    def shape(self) -> int:
        return len(self.volatome_storage)

    @classmethod
    def from_json(cls, filename: str):
        with open(filename, 'r') as f:
            data = json.load(f)
            volatome_void = cls()
            for uid, volatome_data in data.items():
                volatome_data['collection_time'] = pd.to_datetime(volatome_data['collection_time'])
                volatome = Volatome(**volatome_data)
                volatome_void.add_volatome(volatome)
            return volatome_void

    def search(self, **kwargs):
        results = []
        for volatome in self.volatome_storage.values():
            match = True
            for key, value in kwargs.items():
                if hasattr(volatome, key):
                    if getattr(volatome, key) != value:
                        match = False
                elif key == 'targets':
                    if not any(t.short_name == value for t in volatome.targets):
                        match = False
                elif key == 'voc_objects':
                    if not any(v.identifier == value for v in volatome.voc_objects):
                        match = False
            if match:
                results.append(volatome)
        return results

@dataclass
class TargetVoid:
    target_storage: dict = field(default_factory=dict)

    def add_target(self, target: Target):
        """Add a Target object to the storage."""
        self.target_storage[(target.icd_10, target.icd_11)] = target

    def get_target(self, icd_10: str, icd_11: str) -> Optional[Target]:
        """Retrieve a Target object by its icd codes."""
        return self.target_storage.get((icd_10, icd_11), None)

    def remove_target(self, icd_10: str, icd_11: str):
        """Remove a Target object from the storage."""
        if (icd_10, icd_11) in self.target_storage:
            del self.target_storage[(icd_10, icd_11)]

    def to_json(self, filename: str):
        with open(filename, 'w') as f:
            json.dump({(target.icd_10, target.icd_11): target.__dict__ for target in self.target_storage.values()}, f)

    @classmethod
    def from_json(cls, filename: str):
        with open(filename, 'r') as f:
            data = json.load(f)
            target_void = cls()
            for (icd_10, icd_11), target_data in data.items():
                target = Target(**target_data)
                target_void.add_target(target)
            return target_void

    def search(self, short_name: Optional[str] = None, icd_10: Optional[str] = None, icd_11: Optional[str] = None):
        results = []
        for target in self.target_storage.values():
            if (short_name is None or target.short_name == short_name) and \
               (icd_10 is None or target.icd_10 == icd_10) and \
               (icd_11 is None or target.icd_11 == icd_11):
                results.append(target)
        return results

@dataclass
class PersonVoid:
    person_storage: dict = field(default_factory=dict)

    def add_person(self, person: Person):
        """Add a Person object to the storage."""
        self.person_storage[person.person_id] = person

    def get_person(self, person_id: str) -> Optional[Person]:
        """Retrieve a Person object by its person_id."""
        return self.person_storage.get(person_id, None)

    def remove_person(self, person_id: str):
        """Remove a Person object from the storage."""
        if person_id in self.person_storage:
            del self.person_storage[person_id]

    def to_json(self, filename: str):
        with open(filename, 'w') as f:
            json.dump({person_id: person.__dict__ for person_id, person in self.person_storage.items()}, f)

    @classmethod
    def from_json(cls, filename: str):
        with open(filename, 'r') as f:
            data = json.load(f)
            person_void = cls()
            for person_id, person_data in data.items():
                person = Person(**person_data)
                person_void.add_person(person)
            return person_void

    def search(self, person_id: Optional[str] = None):
        results = []
        for person in self.person_storage.values():
            if person_id is None or person.person_id == person_id:
                results.append(person)
        return results

class DataLoader:
    def __init__(self, dataframe: pd.DataFrame):
        self.dataframe = dataframe
        self.volatome_void = VolatomeVoid()
        self.person_void = PersonVoid()
        self.target_void = TargetVoid()

    def load_data(self):
        # Load targets into TargetVoid
        for index, row in self.dataframe.iterrows():
            target = Target(
                short_name=row['target_short_name'],
                icd_10=row['target_icd_10'],
                icd_11=row['target_icd_11']
            )
            self.target_void.add_target(target)

        # Load persons into PersonVoid
        for person_id in self.dataframe['PatientID'].unique():
            person = Person(person_id=str(person_id))
            self.person_void.add_person(person)

        # Load Volatome objects into VolatomeVoid
        for index, row in self.dataframe.iterrows():
            person = self.person_void.get_person(str(row['PatientID']))
            if person:
                # Create VOC objects from the row
                voc_objects = [
                    VOC(identifier=col, value=row[col]) 
                    for col in self.dataframe.columns[:-5]  # Exclude PatientID, ProbeId, DateTime, target_short_name, target_icd_10, target_icd_11
                ]
                
                # Create targets from the target column
                targets = [self.target_void.get_target(row['target_icd_10'], row['target_icd_11'])]

                # Create Volatome object
                volatome = Volatome(
                    unique_id=row['ProbeId'],
                    person=person,
                    collection_time=row['DateTime'],
                    targets=targets,
                    voc_objects=voc_objects
                )
                
                # Add Volatome to VolatomeVoid
                self.volatome_void.add_volatome(volatome)


In [176]:
data_loader1 = DataLoader(df1)  # Replace df1 with your first DataFrame
data_loader2 = DataLoader(df2)  # Replace df2 with your second DataFrame

# Load data into the respective DataLoader objects
data_loader1.load_data()
data_loader2.load_data()

merged_volatome_void = VolatomeVoid()
merged_volatome_void.volatome_storage = {**data_loader1.volatome_void.volatome_storage, **data_loader2.volatome_void.volatome_storage}

In [177]:
merged_volatome_void.shape

79

In [179]:
merged_volatome_void.shape

merged_volatome_void.to_json("merged_volatomes.json")

merged_volatome_void

VolatomeVoid(volatome_storage={'ProbeId_0': Volatome(unique_id='ProbeId_0', person=Person(person_id='17503'), collection_time=Timestamp('2024-08-31 10:10:17.979849'), breath_type='normal', normalization='raw', targets=[Target(short_name='ИБС', icd_10='I20.8', icd_11='BA40.1')], voc_objects=[VOC(identifier='42_03257085331836', value=0.06751194845337952, method='Silantiev_maximization', chemical_formula=None), VOC(identifier='43_04741898699762', value=0.6289853425890384, method='Silantiev_maximization', chemical_formula=None), VOC(identifier='44_01450529149254', value=0.07997909294346803, method='Silantiev_maximization', chemical_formula=None), VOC(identifier='44_0489926172728', value=0.06570503692665408, method='Silantiev_maximization', chemical_formula=None), VOC(identifier='44_99144370755021', value=3.712502472083854, method='Silantiev_maximization', chemical_formula=None), VOC(identifier='45_99197266970869', value=0.06579007695402857, method='Silantiev_maximization', chemical_formula

In [147]:
!pwd

12738.70s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/home/guest


In [ ]:

# Example Usage
if __name__ == "__main__":
    # Sample data as a DataFrame
    data = {
        '42_03257085331836': [0.0347108, 0.936501, 0.0703742, 0.0917151, 0.741693, 0.074873, 0.106864, 0.171386],
        '43_04741898699762': [0.289125, 1.91298, 2.49884, 1.27432, 0.964586, 0.811691, 3.14315, 3.38432],
        '44_01450529149254': [0.0396336, 0.0739828, 0.0789201, 0.0940473, 0.0550934, 0.0709599, 0.0991804, 0.0795997],
        '44_0489926172728': [0.0231194, 0.087051, 0.101689, 0.0762481, 0.0501778, 0.0548712, 0.139365, 0.137908],
        '44_99144370755021': [3.52877, 3.51884, 2.68527, 5.33148, 3.35456, 3.9698, 4.15313, 2.83617],
        'PatientID': [17558, 21569, 21094, 19907, 20195, 21387, 20007, 18380],
        'ProbeId': ['ProbeId_2', 'ProbeId_23', 'ProbeId_74', 'ProbeId_20', 'ProbeId_60', 'ProbeId_71', 'ProbeId_14', 'ProbeId_51'],
        'DateTime': [
            '2024-08-30 18:37:04.301985', '2024-08-30 18:37:04.302000', 
            '2024-08-30 18:37:04.302043', '2024-08-30 18:37:04.301998', 
            '2024-08-30 18:37:04.302033', '2024-08-30 18:37:04.302040', 
            '2024-08-30 18:37:04.301994', '2024-08-30 18:37:04.302026'
        ],
        'target_short_name': ['ИБС', 'ИБС', 'Healthy', 'ИБС', 'Healthy', 'Healthy', 'ИБС', 'Healthy'],
        'target_icd_10': ['I20.8', 'I20.8', 'Healthy', 'I20.8', 'Healthy', 'Healthy', 'I20.8', 'Healthy'],
        'target_icd_11': ['BA40.1', 'BA40.1', 'Healthy', 'BA40.1', 'Healthy', 'Healthy', 'BA40.1', 'Healthy']
    }

    df = pd.DataFrame(data)
    df['DateTime'] = pd.to_datetime(df['DateTime'])  # Convert DateTime to datetime objects

    # Load data using DataLoader
    data_loader = DataLoader(df)
    data_loader.load_data()

    # Print the loaded Volatome objects
    for volatome in data_loader.volatome_void.volatome_storage.values():
        print(volatome)

    # Print the loaded Person objects
    for person in data_loader.person_void.person_storage.values():
        print(person)

    # Print the loaded Target objects
    for target in data_loader.target_void.target_storage.values():
        print(target)

In [ ]:
vocs_names = [x.replace('l0_', '') for x in df.columns if 'l0_' in x][:5]

df_vocs = df[[x for x in df.columns if 'l0_' in x][:5]]
df_vocs.columns = vocs_names

unique_probe_id = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['PatientID'] = df['Cardio_Qvark_ID']
df_vocs['ProbeId'] = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['DateTime'] = [datetime.now() for x in range(df.shape[0])]

y = (df['Myocardial_perfusion_defect_after_stress_ATP'] == 'Yes').astype(float)
df_vocs['target'] =y.replace([0, 1], ['Healthy', 'ИБС'])

X_train, _, y_train, _ = train_test_split(df_vocs.drop('target', axis = 'columns'), y, test_size=.9, random_state=42)
X_train['target'] =y_train.replace([0, 1], ['Healthy', 'ИБС'])

df_vocs = X_train

print(df_vocs.to_markdown())

In [ ]:
vocs_names = [x.replace('l0_', '') for x in df.columns if 'l0_' in x][:5]

df_vocs = df[[x for x in df.columns if 'l0_' in x][:5]]
df_vocs.columns = vocs_names

unique_probe_id = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['PatientID'] = df['Cardio_Qvark_ID']
df_vocs['ProbeId'] = ['ProbeId_' + str(x) for x in range(df.shape[0])]
df_vocs['DateTime'] = [datetime.now() for x in range(df.shape[0])]

y = (df['Myocardial_perfusion_defect_after_stress_ATP'] == 'Yes').astype(float)
df_vocs['target'] =y.replace([0, 1], ['Healthy', 'ИБС'])

X_train, _, y_train, _ = train_test_split(df_vocs.drop('target', axis = 'columns'), y, test_size=.9, random_state=42)
X_train['target'] =y_train.replace([0, 1], ['Healthy', 'ИБС'])

df_vocs = X_train

print(df_vocs.to_markdown())

In [134]:
df

,42_03257085331836,43_04741898699762,44_01450529149254,44_0489926172728,44_99144370755021,PatientID,ProbeId,DateTime,target_short_name,target_icd_10,target_icd_11
0,0.034711,0.289125,0.039634,0.023119,3.52877,17558,ProbeId_2,2024-08-30 18:37:04.301985,ИБС,I20.8,BA40.1
1,0.936501,1.912980,0.073983,0.087051,3.51884,21569,ProbeId_23,2024-08-30 18:37:04.302000,ИБС,I20.8,BA40.1
2,0.070374,2.498840,0.078920,0.101689,2.68527,21094,ProbeId_74,2024-08-30 18:37:04.302043,Healthy,Healthy,Healthy
3,0.091715,1.274320,0.094047,0.076248,5.33148,19907,ProbeId_20,2024-08-30 18:37:04.301998,ИБС,I20.8,BA40.1
4,0.741693,0.964586,0.055093,0.050178,3.35456,20195,ProbeId_60,2024-08-30 18:37:04.302033,Healthy,Healthy,Healthy
5,0.074873,0.811691,0.070960,0.054871,3.96980,21387,ProbeId_71,2024-08-30 18:37:04.302040,Healthy,Healthy,Healthy
6,0.106864,3.143150,0.099180,0.139365,4.15313,20007,ProbeId_14,2024-08-30 18:37:04.301994,ИБС,I20.8,BA40.1
7,0.171386,3.384320,0.079600,0.137908,2.83617,18380,ProbeId_51,2024-08-30 18:37:04.302026,Healthy,Healthy,Healthy


In [74]:
Basheer_void = VolatomeVoid()


# targets = TargetVoid()
# targets.add_target(Target(short_name = "Control", icu_10='Healthy', icu_11='Healthy'))
# targets.add_target(Target(short_name = "ИБС", icu_10='I20.8', icu_11='BA40.1'))


# volatome_void = VolatomeVoid()

# for col in vocs_names:
#     v = VOC(identifier=col, value=df_vocs.loc[0, col])
#     vvocs = vvocs + [v]

/tmp/ipykernel_311230/2876285011.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vocs['PatientID'] = df['Cardio_Qvark_ID']
/tmp/ipykernel_311230/2876285011.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vocs['ProbeId'] = ['ProbeId_' + str(x) for x in range(df.shape[0])]
/tmp/ipykernel_311230/2876285011.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [75]:
df_vocs

,42_03257085331836,43_04741898699762,44_01450529149254,44_0489926172728,44_99144370755021,45_99197266970869,47_04077398412186,48_045099177099885,49_00477743760921,49_99501251131184,...,373_086783934889,374_08597586892273,375_07957930255367,429_0881741701056,430_0853629703972,445_1153231924922,447_1045512664764,PatientID,ProbeId,DateTime
0,0.067512,0.628985,0.079979,0.065705,3.712502,0.065790,0.103783,0.005272,0.046767,0.007219,...,0.004314,0.001512,0.001085,0.001185,0.000422,0.000701,0.000758,17503,ProbeId_0,2024-08-30 17:49:46.882083
1,0.052837,0.293217,0.041497,0.027165,3.342605,0.056691,0.089783,0.002514,0.008929,0.004855,...,0.010301,0.003192,0.001843,0.039475,0.016813,0.027573,0.017263,17494,ProbeId_1,2024-08-30 17:49:46.882090
2,0.034711,0.289125,0.039634,0.023119,3.528768,0.058900,0.210027,0.006351,0.027325,0.005452,...,0.001515,0.000614,0.000415,0.000659,0.000406,0.000730,0.000571,17558,ProbeId_2,2024-08-30 17:49:46.882091
3,0.065292,0.965013,0.055779,0.059098,2.963412,0.052547,0.143320,0.004360,0.017409,0.006269,...,0.006607,0.002000,0.001257,0.001826,0.000944,0.001663,0.000869,17504,ProbeId_3,2024-08-30 17:49:46.882091
4,0.080594,3.142379,0.052536,0.122263,2.635041,0.043910,0.100054,0.002831,0.004470,0.004573,...,0.065896,0.020720,0.011381,0.003077,0.001288,0.002594,0.001416,17698,ProbeId_4,2024-08-30 17:49:46.882092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.070222,1.218192,0.060510,0.058470,3.800003,0.064128,0.162522,0.004768,0.066721,0.009513,...,0.014060,0.004339,0.002725,0.006782,0.003118,0.004354,0.003197,23431,ProbeId_75,2024-08-30 17:49:46.882129
76,0.083748,2.472044,0.110516,0.126304,2.979950,0.058456,8.012591,0.178858,0.030891,0.008964,...,0.030824,0.010092,0.005993,0.014523,0.005712,0.010023,0.006641,22015,ProbeId_76,2024-08-30 17:49:46.882130
77,0.059168,2.266683,0.108345,0.127781,1.556364,0.071685,0.186592,0.005847,0.043078,0.005111,...,0.004727,0.001663,0.001062,0.001498,0.000586,0.000894,0.000546,20283,ProbeId_77,2024-08-30 17:49:46.882131
78,0.288774,1.230780,0.062873,0.050527,1.983028,0.051258,0.217254,0.006029,0.002695,0.002760,...,0.002709,0.001151,0.000797,0.001336,0.000820,0.001174,0.000667,17795,ProbeId_78,2024-08-30 17:49:46.882131
